In [1]:
import cv2
import numpy as np
%matplotlib inline

In [2]:
corner_track_parameter = dict(maxCorners = 5, qualityLevel = 0.3, minDistance = 7, blockSize = 7)
lk_parameter = dict(winSize=(200,200),maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,0.03))

In [3]:
videoCapture = cv2.VideoCapture(0)
_, last_frame = videoCapture.read()
last_frame_gray = cv2.cvtColor(last_frame,cv2.COLOR_BGR2GRAY)

# Detect points
last_points = cv2.goodFeaturesToTrack(last_frame_gray,mask=None,**corner_track_parameter)
mask = np.zeros_like(last_frame)

while 1:
    _, frame = videoCapture.read()
    frame_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    # Optical Flow Pyramid
    now_points, status, error = cv2.calcOpticalFlowPyrLK(last_frame_gray,frame_gray,last_points,None,**lk_parameter)
    
    good_now = now_points[status == 1]
    good_last = last_points[status == 1]
    
    for i,(now,last) in enumerate(zip(good_now,good_last)):
        x_now,y_now = now.ravel()
        x_last,y_last = last.ravel()
        mask = cv2.line(mask,(x_now,y_now),(x_last,y_last),(0,255,0),3)
        frame = cv2.circle(frame,(x_now,y_now),8,(0,0,255),-1)
    
    image = cv2.add(frame,mask)
    cv2.imshow('Camera',image)
    
    if cv2.waitKey(1) == 27:
        break
    
    last_frame_gray = frame_gray.copy()
    last_points = good_now.reshape(-1,1,2)

videoCapture.release()
cv2.destroyAllWindows()